### Creating SQLite database and tables

In [ ]:
import sqlite3

# create and connect to database
conn = sqlite3.connect("instagram_artists_apresearch.db")
cursor = conn.cursor()

cmd = ""

We will have 2 tables, one for the profiles and one for the posts.
Both tables will contain both human and ai content, with a column identifying which they are

In [ ]:
# create table of all profiles
# column is_human is true if profile is a human artist
cmd = """CREATE TABLE PROFILES(
    user_id INT,
    username VARCHAR(30),
    num_followers INT,
    num_posts INT,
    is_human BOOLEAN,
    PRIMARY KEY (user_id)
    );"""
cursor.execute(cmd)

# create table of all posts
cmd = """CREATE TABLE POSTS(
owner_user_id INT,
owner_username VARCHAR(30),
num_likes INT,
num_comments INT,
shortcode VARCHAR(255),
img_url VARCHAR(255),
post_date DATE,
is_reel BOOLEAN,
FOREIGN KEY(owner_user_id) REFERENCES PROFILES(user_id)
);"""
cursor.execute(cmd)

### Adding data into our sql database and tables

In [ ]:
# %pip install instaloader
import instaloader

# Creating an instance of the Instaloader class
bot = instaloader.Instaloader()
# bot.login(user="Your_username",passwd="Your_password") ## Use this to log-in to your account
# NOTE signing in will probably get your account terminated/banned on instagram, so it is safest to use instaloader anonymously

We will create our tables using .csv files. Files must formatted so the first row are column names and thefirst column will not contain information.

#### NOTE Only values outside of the first row and column will be added into the table

In [1]:
# % pip install pandas
import pandas as pd

def add_to_tables(csv_path, is_human_val):
    conn = sqlite3.connect("instagram_artists_apresearch.db")
    cursor = conn.cursor()

    df = pd.read_csv(csv_path)

    past_post = True # set to False if you need to only add posts after a specific post within a profile (will continue adding profiles after)
    past_profile = past_post and True # set to False if you need to only add profiles starting at a specific profile or post
    # if past_post is False, past_profile must be False


    for i in df.index:
        for col in df.columns[1:]:
            curr_post = None
            try:
                if(pd.notna(df.loc[i,col])):
                    ## used to start adding profiles at a profile or post (profile inclusive, post exclusive)
                    # if(str(df.loc[i,col]) == "start_profile_username"):
                    #     past_profile = True
                    if(past_profile):
                        p = instaloader.Profile.from_username(bot.context, df.loc[i,col]) # grab profile from instagram

                        print(f"{p.userid}, {p.username}, {p.followers}, {p.mediacount}, {is_human_val}, {df.iloc[i,0]}") # track current profile

                        # check if profile does not already exist in table
                        cursor.execute("SELECT 1 FROM PROFILES WHERE USER_ID = {}".format(p.userid))
                        existing_row = cursor.fetchone()

                        if not existing_row:
                            # insert the row
                            cmd = """INSERT INTO PROFILES(USER_ID, USERNAME, NUM_FOLLOWERS, NUM_POSTS, IS_HUMAN, GENRE) 
                            VALUES ({}, "{}", {}, {}, {}, "{}");""".format(
                                p.userid, p.username, p.followers, p.mediacount, is_human_val, df.iloc[i,0]
                            )
                            cursor.execute(cmd)
                        
                        for post in p.get_posts():
                            if(past_post):
                                curr_post = post.shortcode # keep track of current post which was added
                                cmd = """INSERT INTO POSTS(OWNER_USER_ID, OWNER_USERNAME, NUM_LIKES, NUM_COMMENTS, SHORTCODE, IMG_URL, POST_DATE, IS_REEL, IS_HUMAN, GENRE) 
                                VALUES ({}, "{}", {}, {}, "{}", "{}", "{}", {}, {}, "{}");""".format(
                                    post.owner_id, post.owner_username, post.likes, post.comments, post.shortcode, post.url, post.date_utc, False, is_human_val, df.iloc[i,0]
                                )
                                cursor.execute(cmd)

                            ## used to start adding after a specific post (post exclusive). this post should already be included in the table
                            ## uses post shortcode as identifier
                            # if (post.shortcode == "xxxxxxxxxxx"):
                            #     past_post = True
            # exception handling mainly for profiles not found, faulty networks connections, or instagram blocking us temporarily
            # this helps keep track of and continue where we left off, which is why we have the past_profile and past_post
            except Exception as e:
                conn.commit()
                conn.close()
                print(f"Error with {df.loc[i,col]}, after {curr_post} : {e}")
                return
    # IMPORTANT finally save all changes to database, changes are not committed every insert
    conn.commit()
    conn.close()

In [ ]:
add_to_tables("HA.csv", True) # add human profiles and posts into tables
add_to_tables("AIA.csv", False) # add AI profiles and posts into tables